In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
import gc
from tqdm import tqdm, tqdm_notebook

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [3]:
basic_fea = ['orderid','city','countryid','hotel','zone','room','isholdroom','ordadvanceday','masterbasicroomid','masterhotelid','supplierid','isvendor','isebookinghtl','hotelstar','supplierchannel','orderdate_mon','arrival_mon','etd_mon','orderdate_day','arrival_day','etd_day','totalrooms_x','glon','glat','totalrooms_y','actual_day','order_day','is_equal']
basic_fea_noid = ['isholdroom','ordadvanceday','isvendor','isebookinghtl','hotelstar','supplierchannel','orderdate_mon','arrival_mon','etd_mon','orderdate_day','arrival_day','etd_day','totalrooms_x','glon','glat','totalrooms_y','actual_day','order_day','is_equal']
target = ['noroom']
# all_fea = list(set(train.columns) - set(basic_fea + ['noroom', 'hasroom']))

In [4]:
good_fea = ['hotel_hot','pos_hotel_isholdroom_mean_sub','pos_hotel_orderdate_mon_max','pos_hotel_ordadvanceday_median','pos_hotel_arrival_mon_mean','pos_hotel_isvendor_mean_sub','pos_hotel_isholdroom_mean','pos_hotel_ordadvanceday_max','pos_hotel_supplierchannel_mean_sub','pos_hotel_isebookinghtl_median_sub','pos_hotel_supplierchannel_median_sub','pos_hotel_hotelstar_median_sub','pos_hotel_hotelbelongto_mean_sub','pos_hotel_hotelbelongto_median_sub','pos_hotel_isebookinghtl_mean_sub','pos_hotel_hotelstar_mean_sub','nag_hotel_etd_mon_var','nag_masterhotelid_etd_mon_max','nag_hotel_orderdate_mon_mean','pos_hotel_etd_mon_max','pos_hotel_etd_mon_min','pos_hotel_orderdate_mon_median','nag_masterhotelid_arrival_mon_max','pos_hotel_orderdate_mon_mean_sub','pos_hotel_arrival_mon_max','pos_hotel_etd_mon_mean_sub','pos_hotel_orderdate_mon_median_sub','pos_hotel_orderdate_mon_var','pos_hotel_isholdroom_median','pos_hotel_ordadvanceday_mean','supplierid_his_daysum','hotel_num','pos_hotel_hotelbelongto_median','pos_hotel_hotelbelongto_min','pos_hotel_hotelbelongto_mean','nag_hotel_ordadvanceday_max','nag_hotel_isholdroom_min','pos_hotel_isholdroom_median_sub','pos_hotel_ordadvanceday_var','nag_hotel_arrival_mon_max','nag_hotel_isvendor_var','supplierid_num','nag_hotel_orderdate_mon_var','pos_hotel_isvendor_min','nag_hotel_ordadvanceday_var','nag_masterhotelid_orderdate_mon_median_sub','nag_masterhotelid_etd_mon_var','pos_hotel_isholdroom_var','nag_hotel_isebookinghtl_mean_sub','nag_hotel_supplierchannel_median_sub','nag_hotel_hotelstar_median_sub','nag_hotel_hotelbelongto_median_sub','nag_hotel_isebookinghtl_median_sub','nag_hotel_supplierchannel_mean_sub','nag_hotel_hotelbelongto_mean_sub','nag_hotel_hotelstar_mean_sub','nag_hotel_supplierchannel_max','nag_hotel_supplierchannel_mean','nag_hotel_supplierchannel_median','nag_hotel_supplierchannel_min','nag_hotel_isholdroom_var','nag_masterhotelid_isvendor_mean_sub','nag_hotel_orderdate_mon_mean_sub','pos_hotel_etd_mon_median','nag_hotel_etd_mon_max','pos_hotel_isholdroom_max','supplierid_hot','nag_hotel_etd_mon_median','hotel_his_daysum','nag_masterhotelid_supplierchannel_mean','nag_hotel_arrival_mon_median','pos_supplierid_ordadvanceday_mean','nag_supplierid_ordadvanceday_median','pos_hotel_etd_mon_mean','pos_hotel_arrival_mon_min','nag_hotel_orderdate_mon_median','pos_hotel_arrival_mon_median_sub','pos_hotel_arrival_mon_var','pos_hotel_isholdroom_min','nag_hotel_arrival_mon_mean','pos_hotel_etd_mon_median_sub','pos_masterhotelid_hotelbelongto_var','nag_hotel_isholdroom_max','pos_hotel_supplierchannel_median','pos_hotel_supplierchannel_mean','pos_hotel_supplierchannel_min','pos_hotel_supplierchannel_max','pos_hotel_isvendor_mean','pos_hotel_ordadvanceday_min','pos_hotel_orderdate_mon_min','pos_hotel_isvendor_median_sub','nag_hotel_orderdate_mon_min','pos_hotel_isvendor_median','nag_hotel_arrival_mon_var','nag_hotel_isvendor_max','pos_room_isholdroom_max','pos_masterhotelid_etd_mon_max','nag_masterhotelid_etd_mon_median','nag_masterhotelid_ordadvanceday_max','nag_hotel_orderdate_mon_max','pos_masterhotelid_isvendor_var','pos_masterhotelid_supplierchannel_var','nag_masterhotelid_supplierchannel_mean_sub','nag_hotel_ordadvanceday_median_sub','room_hot','pos_masterhotelid_hotelbelongto_mean','nag_hotel_isholdroom_mean','pos_hotel_arrival_mon_median','nag_hotel_arrival_mon_median_sub','pos_masterhotelid_supplierchannel_mean','nag_masterhotelid_isebookinghtl_mean','nag_masterhotelid_arrival_mon_var','pos_hotel_hotelstar_median','pos_hotel_hotelstar_max','pos_hotel_hotelstar_mean','pos_hotel_hotelstar_min','pos_hotel_etd_mon_var','hotelorder_maxdate_sub','nag_hotel_arrival_mon_min','pos_supplierid_arrival_mon_max','pos_hotel_isvendor_max','nag_masterhotelid_orderdate_mon_mean','nag_masterhotelid_hotelbelongto_var','nag_masterhotelid_ordadvanceday_var','pos_hotel_isvendor_var','pos_hotel_isebookinghtl_median','pos_hotel_isebookinghtl_min','pos_hotel_isebookinghtl_mean','pos_hotel_isebookinghtl_max','nag_masterhotelid_isholdroom_min','pos_supplierid_orderdate_mon_mean','nag_masterhotelid_isebookinghtl_median','pos_hotel_orderdate_mon_mean','nag_masterhotelid_arrival_mon_median','nag_masterhotelid_isvendor_mean','pos_room_isholdroom_median_sub','pos_hotel_ordadvanceday_mean_sub','pos_masterhotelid_ordadvanceday_max','nag_hotel_isvendor_mean','pos_hotel_ordadvanceday_median_sub','pos_supplierid_orderdate_mon_mean_sub','pos_masterhotelid_supplierchannel_median_sub','pos_masterhotelid_isvendor_mean_sub','pos_masterhotelid_orderdate_mon_mean_sub','nag_hotel_isholdroom_mean_sub','pos_masterhotelid_isebookinghtl_mean','nag_supplierid_etd_mon_mean','pos_supplierid_arrival_mon_mean','nag_hotel_isebookinghtl_var','nag_hotel_hotelbelongto_var','nag_hotel_hotelstar_var','nag_hotel_supplierchannel_var','nag_hotel_isvendor_median_sub','nag_supplierid_orderdate_mon_var','nag_masterhotelid_orderdate_mon_mean_sub','pos_masterhotelid_isvendor_max','nag_masterhotelid_hotelbelongto_mean','nag_masterhotelid_arrival_mon_min','pos_supplierid_hotelstar_var','nag_hotel_isebookinghtl_median','nag_hotel_isebookinghtl_min','nag_hotel_isebookinghtl_max','nag_hotel_isebookinghtl_mean','pos_masterhotelid_isebookinghtl_mean_sub','nag_hotel_ordadvanceday_median','masterhotelid_hot','pos_room_isholdroom_mean','nag_masterhotelid_arrival_mon_median_sub','nag_masterhotelid_orderdate_mon_var','nag_masterhotelid_hotelbelongto_mean_sub','pos_city_hotelbelongto_median','pos_masterhotelid_arrival_mon_min','supplieridorder_mindate_sub','pos_city_hotelbelongto_mean','pos_masterhotelid_orderdate_mon_mean','nag_masterhotelid_isholdroom_median_sub','nag_masterbasicroomid_hotelbelongto_mean','nag_supplierid_orderdate_mon_median_sub','pos_room_supplierchannel_mean','pos_room_supplierchannel_median','pos_room_supplierchannel_max','pos_room_supplierchannel_min','nag_hotel_isvendor_median','nag_masterhotelid_hotelstar_max','nag_hotel_arrival_mon_mean_sub','nag_city_isholdroom_var','pos_masterhotelid_etd_mon_min','nag_supplierid_hotelstar_var','pos_masterhotelid_isvendor_median_sub','nag_masterhotelid_hotelstar_mean_sub','pos_countryid_hotelstar_median_sub','nag_supplierid_isvendor_median','pos_masterhotelid_ordadvanceday_mean','pos_supplierid_hotelbelongto_min','nag_supplierid_isholdroom_mean_sub','pos_countryid_hotelbelongto_mean_sub','nag_masterhotelid_etd_mon_min','nag_city_arrival_mon_median_sub','pos_countryid_isvendor_mean_sub','pos_hotel_isebookinghtl_var','pos_hotel_supplierchannel_var','pos_hotel_hotelstar_var','pos_hotel_hotelbelongto_var','pos_masterbasicroomid_isholdroom_median','nag_masterhotelid_ordadvanceday_median_sub','pos_supplierid_ordadvanceday_median','nag_masterbasicroomid_arrival_mon_var','pos_countryid_hotelstar_mean_sub','nag_hotel_etd_mon_median_sub','pos_masterhotelid_isvendor_mean','pos_supplierid_etd_mon_mean','pos_masterhotelid_isvendor_median','pos_masterhotelid_hotelbelongto_mean_sub','nag_masterhotelid_supplierchannel_min','pos_room_orderdate_mon_max','pos_supplierid_hotelstar_mean','nag_supplierid_hotelstar_median','pos_city_hotelstar_mean','pos_room_isholdroom_median','supplierchannel_nag_hot','supplierchannel_pos_hot','nag_masterhotelid_orderdate_mon_median','pos_supplierid_orderdate_mon_median_sub','pos_masterhotelid_etd_mon_var','nag_masterhotelid_isholdroom_median','pos_countryid_isebookinghtl_median_sub','nag_masterhotelid_orderdate_mon_min','nag_city_orderdate_mon_median_sub','pos_masterbasicroomid_isebookinghtl_var','nag_masterbasicroomid_hotelbelongto_median_sub','nag_hotel_hotelbelongto_median','nag_hotel_hotelbelongto_max','nag_hotel_hotelbelongto_min','nag_hotel_hotelbelongto_mean','pos_masterhotelid_ordadvanceday_median','pos_supplierid_isholdroom_mean','pos_masterhotelid_supplierchannel_median','pos_masterhotelid_hotelbelongto_max','nag_masterhotelid_isebookinghtl_mean_sub','pos_room_isebookinghtl_median_sub','pos_room_supplierchannel_mean_sub','pos_room_hotelstar_median_sub','pos_room_isvendor_mean_sub','pos_room_hotelbelongto_mean_sub','pos_room_hotelbelongto_median_sub','pos_room_hotelstar_mean_sub','pos_room_isvendor_median_sub','pos_room_isebookinghtl_mean_sub','pos_room_supplierchannel_median_sub','nag_masterhotelid_hotelbelongto_max','nag_supplierid_isebookinghtl_mean','nag_supplierid_arrival_mon_var','hotelorder_mindate_sub','nag_supplierid_arrival_mon_median','pos_masterhotelid_isebookinghtl_median_sub','nag_city_arrival_mon_max','nag_supplierid_etd_mon_var','pos_room_etd_mon_max','nag_supplierid_orderdate_mon_mean_sub','nag_masterhotelid_ordadvanceday_mean_sub','nag_room_isebookinghtl_median_sub','nag_room_supplierchannel_median_sub','nag_room_hotelstar_mean_sub','nag_room_hotelbelongto_mean_sub','nag_room_hotelstar_median_sub','nag_room_supplierchannel_mean_sub','nag_room_isvendor_median_sub','nag_room_isvendor_mean_sub','nag_room_isebookinghtl_mean_sub','nag_room_hotelbelongto_median_sub','pos_countryid_hotelstar_mean','nag_city_arrival_mon_var','nag_masterhotelid_hotelbelongto_min','pos_countryid_ordadvanceday_median_sub','pos_supplierid_hotelbelongto_var','nag_room_etd_mon_median','pos_masterhotelid_hotelbelongto_median','cityorder_mindate_sub','nag_masterhotelid_isholdroom_mean','pos_masterbasicroomid_etd_mon_var','pos_room_arrival_mon_median_sub','masterhotelid_num','nag_masterhotelid_isholdroom_mean_sub','nag_supplierid_ordadvanceday_var','nag_masterhotelid_hotelbelongto_median_sub','nag_countryid_isholdroom_var','nag_supplierid_ordadvanceday_mean','pos_masterhotelid_isvendor_min','nag_city_orderdate_mon_median','nag_countryid_supplierchannel_mean_sub','pos_masterbasicroomid_ordadvanceday_mean','nag_masterbasicroomid_supplierchannel_median','pos_masterhotelid_supplierchannel_mean_sub','pos_supplierid_ordadvanceday_var','nag_masterhotelid_hotelstar_median_sub','pos_masterhotelid_orderdate_mon_max','pos_supplierid_ordadvanceday_max','pos_masterhotelid_arrival_mon_max','nag_countryid_isebookinghtl_mean_sub','nag_countryid_isholdroom_mean','pos_supplierid_etd_mon_var','pos_masterhotelid_isholdroom_median_sub','pos_city_ordadvanceday_var','pos_masterbasicroomid_isebookinghtl_mean_sub','pos_supplierid_arrival_mon_var','pos_supplierid_isholdroom_var','pos_countryid_hotelbelongto_median_sub','nag_masterbasicroomid_hotelbelongto_max','pos_room_etd_mon_median_sub','nag_masterhotelid_isebookinghtl_median_sub','nag_city_isvendor_median_sub','nag_hotel_ordadvanceday_mean','pos_room_hotelbelongto_median','pos_room_hotelbelongto_min','pos_room_hotelbelongto_max','pos_room_hotelbelongto_mean','pos_supplierid_etd_mon_min','pos_countryid_isebookinghtl_median','nag_supplierid_orderdate_mon_median','pos_masterbasicroomid_arrival_mon_median','pos_city_arrival_mon_mean_sub','pos_countryid_supplierchannel_min','nag_masterhotelid_supplierchannel_max','pos_room_etd_mon_median','nag_masterhotelid_isebookinghtl_max','nag_masterhotelid_hotelstar_median','pos_masterhotelid_hotelbelongto_median_sub','nag_masterhotelid_etd_mon_mean','pos_countryid_isvendor_min','room_num','pos_masterhotelid_isebookinghtl_median','pos_masterhotelid_hotelstar_max','nag_supplierid_supplierchannel_mean','nag_hotel_ordadvanceday_mean_sub','nag_masterhotelid_supplierchannel_median_sub','pos_city_ordadvanceday_median_sub','pos_masterbasicroomid_hotelbelongto_mean_sub','pos_masterbasicroomid_isholdroom_median_sub','nag_city_ordadvanceday_max','nag_masterhotelid_isholdroom_var','pos_city_etd_mon_mean','pos_room_arrival_mon_median','nag_supplierid_isvendor_min','pos_supplierid_etd_mon_median','nag_masterhotelid_ordadvanceday_mean','pos_masterhotelid_hotelbelongto_min','pos_masterhotelid_orderdate_mon_var','pos_masterhotelid_orderdate_mon_median','nag_masterbasicroomid_isebookinghtl_max','nag_city_hotelstar_mean','hotelstar_nag_hot','nag_room_isvendor_median','nag_room_isvendor_min','nag_room_isvendor_mean','nag_room_isvendor_max','zq_num','pos_room_isebookinghtl_min','pos_room_isebookinghtl_max','pos_room_isebookinghtl_mean','pos_room_isebookinghtl_median','pos_supplierid_isvendor_min','pos_masterbasicroomid_etd_mon_median','pos_city_etd_mon_min','pos_masterbasicroomid_ordadvanceday_median_sub','nag_masterbasicroomid_arrival_mon_min','nag_city_isholdroom_max','pos_room_etd_mon_min','nag_countryid_hotelbelongto_mean_sub','pos_supplierid_orderdate_mon_max','pos_room_hotelstar_mean','pos_room_hotelstar_median','pos_room_hotelstar_max','pos_room_hotelstar_min','nag_hotel_orderdate_mon_median_sub','nag_masterhotelid_supplierchannel_var','pos_supplierid_isebookinghtl_mean','nag_masterbasicroomid_hotelstar_max','nag_masterbasicroomid_hotelstar_median_sub','nag_masterhotelid_orderdate_mon_max','pos_masterbasicroomid_isebookinghtl_median_sub','pos_countryid_hotelbelongto_var','nag_countryid_hotelstar_median','pos_supplierid_isholdroom_median','nag_supplierid_hotelbelongto_mean']
len(good_fea)
# 20 50 100 210 290

384

In [5]:
good_fea = ['pos_hotel_isholdroom_mean_sub','pos_hotel_orderdate_mon_max','pos_hotel_ordadvanceday_median','pos_hotel_arrival_mon_mean','pos_hotel_isvendor_mean_sub','pos_hotel_isholdroom_mean','pos_hotel_ordadvanceday_max','pos_hotel_supplierchannel_mean_sub','pos_hotel_isebookinghtl_median_sub','pos_hotel_supplierchannel_median_sub','pos_hotel_hotelstar_median_sub','pos_hotel_hotelbelongto_mean_sub','pos_hotel_hotelbelongto_median_sub','pos_hotel_isebookinghtl_mean_sub','pos_hotel_hotelstar_mean_sub','nag_hotel_etd_mon_var','nag_masterhotelid_etd_mon_max','nag_hotel_orderdate_mon_mean','pos_hotel_etd_mon_max','pos_hotel_etd_mon_min','pos_hotel_orderdate_mon_median','nag_masterhotelid_arrival_mon_max','pos_hotel_orderdate_mon_mean_sub','pos_hotel_arrival_mon_max','pos_hotel_etd_mon_mean_sub','pos_hotel_orderdate_mon_median_sub','pos_hotel_orderdate_mon_var','pos_hotel_isholdroom_median','pos_hotel_ordadvanceday_mean','supplierid_his_daysum','hotel_num','pos_hotel_hotelbelongto_median','pos_hotel_hotelbelongto_min','pos_hotel_hotelbelongto_mean','nag_hotel_ordadvanceday_max','nag_hotel_isholdroom_min','pos_hotel_isholdroom_median_sub','pos_hotel_ordadvanceday_var','nag_hotel_arrival_mon_max','nag_hotel_isvendor_var','supplierid_num','nag_hotel_orderdate_mon_var','pos_hotel_isvendor_min','nag_hotel_ordadvanceday_var','nag_masterhotelid_orderdate_mon_median_sub','nag_masterhotelid_etd_mon_var','pos_hotel_isholdroom_var','nag_hotel_isebookinghtl_mean_sub','nag_hotel_supplierchannel_median_sub','nag_hotel_hotelstar_median_sub','nag_hotel_hotelbelongto_median_sub','nag_hotel_isebookinghtl_median_sub','nag_hotel_supplierchannel_mean_sub','nag_hotel_hotelbelongto_mean_sub','nag_hotel_hotelstar_mean_sub','nag_hotel_supplierchannel_max','nag_hotel_supplierchannel_mean','nag_hotel_supplierchannel_median','nag_hotel_supplierchannel_min','nag_hotel_isholdroom_var','nag_masterhotelid_isvendor_mean_sub','nag_hotel_orderdate_mon_mean_sub','pos_hotel_etd_mon_median','nag_hotel_etd_mon_max','pos_hotel_isholdroom_max','supplierid_hot','hotel_his_daysum','nag_masterhotelid_supplierchannel_mean','nag_hotel_arrival_mon_median','pos_supplierid_ordadvanceday_mean','nag_supplierid_ordadvanceday_median','pos_hotel_etd_mon_mean','pos_hotel_arrival_mon_min','nag_hotel_orderdate_mon_median','pos_hotel_arrival_mon_median_sub','pos_hotel_arrival_mon_var','pos_hotel_isholdroom_min','nag_hotel_arrival_mon_mean','pos_hotel_etd_mon_median_sub','pos_masterhotelid_hotelbelongto_var','nag_hotel_isholdroom_max','pos_hotel_supplierchannel_median','pos_hotel_supplierchannel_mean','pos_hotel_supplierchannel_min','pos_hotel_supplierchannel_max','pos_hotel_isvendor_mean','pos_hotel_ordadvanceday_min','pos_hotel_orderdate_mon_min','pos_hotel_isvendor_median_sub','nag_hotel_orderdate_mon_min','pos_hotel_isvendor_median','nag_hotel_arrival_mon_var','nag_hotel_isvendor_max','pos_room_isholdroom_max','pos_masterhotelid_etd_mon_max','nag_masterhotelid_etd_mon_median','nag_masterhotelid_ordadvanceday_max','nag_hotel_orderdate_mon_max','pos_masterhotelid_isvendor_var','pos_masterhotelid_supplierchannel_var','nag_masterhotelid_supplierchannel_mean_sub','nag_hotel_ordadvanceday_median_sub','room_hot','pos_masterhotelid_hotelbelongto_mean','nag_hotel_isholdroom_mean','pos_hotel_arrival_mon_median','nag_hotel_arrival_mon_median_sub','pos_masterhotelid_supplierchannel_mean','nag_masterhotelid_isebookinghtl_mean','nag_masterhotelid_arrival_mon_var','pos_hotel_hotelstar_median','pos_hotel_hotelstar_max','pos_hotel_hotelstar_mean','pos_hotel_hotelstar_min','pos_hotel_etd_mon_var','nag_hotel_arrival_mon_min','pos_supplierid_arrival_mon_max','pos_hotel_isvendor_max','nag_masterhotelid_orderdate_mon_mean','nag_masterhotelid_hotelbelongto_var','nag_masterhotelid_ordadvanceday_var','pos_hotel_isvendor_var','pos_hotel_isebookinghtl_median','pos_hotel_isebookinghtl_min','pos_hotel_isebookinghtl_mean','pos_hotel_isebookinghtl_max','nag_masterhotelid_isholdroom_min','pos_supplierid_orderdate_mon_mean','nag_masterhotelid_isebookinghtl_median','pos_hotel_orderdate_mon_mean','nag_masterhotelid_isvendor_mean','pos_room_isholdroom_median_sub','pos_hotel_ordadvanceday_mean_sub','pos_masterhotelid_ordadvanceday_max','nag_hotel_isvendor_mean','pos_hotel_ordadvanceday_median_sub','pos_supplierid_orderdate_mon_mean_sub','pos_masterhotelid_supplierchannel_median_sub','pos_masterhotelid_isvendor_mean_sub','pos_masterhotelid_orderdate_mon_mean_sub','nag_hotel_isholdroom_mean_sub','pos_masterhotelid_isebookinghtl_mean','nag_supplierid_etd_mon_mean','pos_supplierid_arrival_mon_mean','nag_hotel_isebookinghtl_var','nag_hotel_hotelbelongto_var','nag_hotel_hotelstar_var','nag_hotel_supplierchannel_var','nag_hotel_isvendor_median_sub','nag_supplierid_orderdate_mon_var','nag_masterhotelid_orderdate_mon_mean_sub','pos_masterhotelid_isvendor_max','nag_masterhotelid_hotelbelongto_mean','nag_masterhotelid_arrival_mon_min','pos_supplierid_hotelstar_var','nag_hotel_isebookinghtl_median','nag_hotel_isebookinghtl_min','nag_hotel_isebookinghtl_max','nag_hotel_isebookinghtl_mean','pos_masterhotelid_isebookinghtl_mean_sub','nag_hotel_ordadvanceday_median','masterhotelid_hot','pos_room_isholdroom_mean','nag_masterhotelid_arrival_mon_median_sub','nag_masterhotelid_orderdate_mon_var','nag_masterhotelid_hotelbelongto_mean_sub','pos_city_hotelbelongto_median','pos_masterhotelid_arrival_mon_min','supplieridorder_mindate_sub','pos_city_hotelbelongto_mean','pos_masterhotelid_orderdate_mon_mean','nag_masterhotelid_isholdroom_median_sub','nag_masterbasicroomid_hotelbelongto_mean','nag_supplierid_orderdate_mon_median_sub','pos_room_supplierchannel_mean','pos_room_supplierchannel_median','pos_room_supplierchannel_max','pos_room_supplierchannel_min','nag_hotel_isvendor_median','nag_masterhotelid_hotelstar_max','nag_hotel_arrival_mon_mean_sub','nag_city_isholdroom_var','pos_masterhotelid_etd_mon_min','nag_supplierid_hotelstar_var','pos_masterhotelid_isvendor_median_sub','nag_masterhotelid_hotelstar_mean_sub','pos_countryid_hotelstar_median_sub','nag_supplierid_isvendor_median','pos_masterhotelid_ordadvanceday_mean','pos_supplierid_hotelbelongto_min','nag_supplierid_isholdroom_mean_sub','pos_countryid_hotelbelongto_mean_sub','nag_masterhotelid_etd_mon_min','nag_city_arrival_mon_median_sub','pos_countryid_isvendor_mean_sub','pos_hotel_isebookinghtl_var','pos_hotel_supplierchannel_var','pos_hotel_hotelstar_var','pos_hotel_hotelbelongto_var','pos_masterbasicroomid_isholdroom_median','nag_masterhotelid_ordadvanceday_median_sub','pos_supplierid_ordadvanceday_median','nag_masterbasicroomid_arrival_mon_var','pos_countryid_hotelstar_mean_sub','nag_hotel_etd_mon_median_sub','pos_masterhotelid_isvendor_mean','pos_supplierid_etd_mon_mean','pos_masterhotelid_hotelbelongto_mean_sub','nag_masterhotelid_supplierchannel_min','pos_room_orderdate_mon_max','pos_supplierid_hotelstar_mean','nag_supplierid_hotelstar_median','pos_city_hotelstar_mean','pos_room_isholdroom_median','supplierchannel_nag_hot','supplierchannel_pos_hot','nag_masterhotelid_orderdate_mon_median','pos_supplierid_orderdate_mon_median_sub','pos_masterhotelid_etd_mon_var','nag_masterhotelid_isholdroom_median','pos_countryid_isebookinghtl_median_sub','nag_masterhotelid_orderdate_mon_min','nag_city_orderdate_mon_median_sub','pos_masterbasicroomid_isebookinghtl_var','nag_masterbasicroomid_hotelbelongto_median_sub','nag_hotel_hotelbelongto_median','nag_hotel_hotelbelongto_max','nag_hotel_hotelbelongto_min','nag_hotel_hotelbelongto_mean','pos_masterhotelid_ordadvanceday_median','pos_supplierid_isholdroom_mean','pos_masterhotelid_supplierchannel_median','pos_masterhotelid_hotelbelongto_max','nag_masterhotelid_isebookinghtl_mean_sub','pos_room_isebookinghtl_median_sub','pos_room_supplierchannel_mean_sub','pos_room_hotelstar_median_sub','pos_room_isvendor_mean_sub','pos_room_hotelbelongto_mean_sub','pos_room_hotelbelongto_median_sub','pos_room_hotelstar_mean_sub','pos_room_isvendor_median_sub','pos_room_isebookinghtl_mean_sub','pos_room_supplierchannel_median_sub','nag_masterhotelid_hotelbelongto_max','nag_supplierid_isebookinghtl_mean','nag_supplierid_arrival_mon_var','hotelorder_mindate_sub','nag_supplierid_arrival_mon_median','pos_masterhotelid_isebookinghtl_median_sub','nag_city_arrival_mon_max','nag_supplierid_etd_mon_var','pos_room_etd_mon_max','nag_supplierid_orderdate_mon_mean_sub','nag_masterhotelid_ordadvanceday_mean_sub','nag_room_isebookinghtl_median_sub','nag_room_supplierchannel_median_sub','nag_room_hotelstar_mean_sub','nag_room_hotelbelongto_mean_sub','nag_room_hotelstar_median_sub','nag_room_supplierchannel_mean_sub','nag_room_isvendor_median_sub','nag_room_isvendor_mean_sub','nag_room_isebookinghtl_mean_sub','nag_room_hotelbelongto_median_sub','pos_countryid_hotelstar_mean','nag_city_arrival_mon_var','nag_masterhotelid_hotelbelongto_min','pos_countryid_ordadvanceday_median_sub','pos_supplierid_hotelbelongto_var','nag_room_etd_mon_median','pos_masterhotelid_hotelbelongto_median','cityorder_mindate_sub','nag_masterhotelid_isholdroom_mean','pos_masterbasicroomid_etd_mon_var','pos_room_arrival_mon_median_sub','masterhotelid_num','nag_masterhotelid_isholdroom_mean_sub','nag_supplierid_ordadvanceday_var','nag_masterhotelid_hotelbelongto_median_sub','nag_countryid_isholdroom_var','nag_supplierid_ordadvanceday_mean','pos_masterhotelid_isvendor_min','nag_city_orderdate_mon_median','nag_countryid_supplierchannel_mean_sub','pos_masterbasicroomid_ordadvanceday_mean','nag_masterbasicroomid_supplierchannel_median','pos_masterhotelid_supplierchannel_mean_sub','pos_supplierid_ordadvanceday_var','nag_masterhotelid_hotelstar_median_sub','pos_masterhotelid_orderdate_mon_max','pos_supplierid_ordadvanceday_max','pos_masterhotelid_arrival_mon_max','nag_countryid_isebookinghtl_mean_sub','nag_countryid_isholdroom_mean','pos_supplierid_etd_mon_var','pos_masterhotelid_isholdroom_median_sub','pos_city_ordadvanceday_var','pos_masterbasicroomid_isebookinghtl_mean_sub','pos_supplierid_arrival_mon_var','pos_supplierid_isholdroom_var','pos_countryid_hotelbelongto_median_sub','nag_masterbasicroomid_hotelbelongto_max','pos_room_etd_mon_median_sub','nag_masterhotelid_isebookinghtl_median_sub','nag_city_isvendor_median_sub','nag_hotel_ordadvanceday_mean','pos_room_hotelbelongto_median','pos_room_hotelbelongto_min','pos_room_hotelbelongto_max','pos_room_hotelbelongto_mean','pos_supplierid_etd_mon_min','pos_countryid_isebookinghtl_median','nag_supplierid_orderdate_mon_median','pos_masterbasicroomid_arrival_mon_median','pos_city_arrival_mon_mean_sub','pos_countryid_supplierchannel_min','nag_masterhotelid_supplierchannel_max','pos_room_etd_mon_median','nag_masterhotelid_isebookinghtl_max','nag_masterhotelid_hotelstar_median','pos_masterhotelid_hotelbelongto_median_sub','nag_masterhotelid_etd_mon_mean','pos_countryid_isvendor_min','room_num','pos_masterhotelid_isebookinghtl_median','pos_masterhotelid_hotelstar_max','nag_supplierid_supplierchannel_mean','nag_hotel_ordadvanceday_mean_sub','nag_masterhotelid_supplierchannel_median_sub','pos_city_ordadvanceday_median_sub','pos_masterbasicroomid_hotelbelongto_mean_sub','pos_masterbasicroomid_isholdroom_median_sub','nag_city_ordadvanceday_max','nag_masterhotelid_isholdroom_var','pos_city_etd_mon_mean','pos_room_arrival_mon_median','nag_supplierid_isvendor_min','pos_supplierid_etd_mon_median','nag_masterhotelid_ordadvanceday_mean','pos_masterhotelid_hotelbelongto_min','pos_masterhotelid_orderdate_mon_var','pos_masterhotelid_orderdate_mon_median','nag_masterbasicroomid_isebookinghtl_max','nag_city_hotelstar_mean','hotelstar_nag_hot','nag_room_isvendor_median','nag_room_isvendor_min','nag_room_isvendor_mean','nag_room_isvendor_max','zq_num','pos_room_isebookinghtl_min','pos_room_isebookinghtl_max','pos_room_isebookinghtl_mean','pos_room_isebookinghtl_median','pos_supplierid_isvendor_min','pos_masterbasicroomid_etd_mon_median','pos_city_etd_mon_min','pos_masterbasicroomid_ordadvanceday_median_sub','nag_masterbasicroomid_arrival_mon_min','nag_city_isholdroom_max','pos_room_etd_mon_min','nag_countryid_hotelbelongto_mean_sub','pos_supplierid_orderdate_mon_max','pos_room_hotelstar_mean','pos_room_hotelstar_median','pos_room_hotelstar_max','pos_room_hotelstar_min','nag_hotel_orderdate_mon_median_sub','nag_masterhotelid_supplierchannel_var','pos_supplierid_isebookinghtl_mean','nag_masterbasicroomid_hotelstar_max','nag_masterbasicroomid_hotelstar_median_sub','nag_masterhotelid_orderdate_mon_max','pos_masterbasicroomid_isebookinghtl_median_sub','pos_countryid_hotelbelongto_var','nag_countryid_hotelstar_median','pos_supplierid_isholdroom_median','nag_supplierid_hotelbelongto_mean']
len(good_fea)
# 20 50 90 180

379

In [6]:
train = pd.read_csv('./features/feature_zero.csv', usecols=basic_fea+good_fea+target+['arrival'])

train.shape
train.head()

(331873, 409)

,orderid,city,countryid,hotel,zone,room,isholdroom,arrival,ordadvanceday,masterbasicroomid,masterhotelid,supplierid,isvendor,isebookinghtl,hotelstar,supplierchannel,noroom,orderdate_mon,arrival_mon,etd_mon,orderdate_day,arrival_day,etd_day,totalrooms_x,glon,glat,totalrooms_y,actual_day,order_day,is_equal,room_hot,masterhotelid_hot,supplierid_hot,hotelstar_nag_hot,supplierchannel_pos_hot,supplierchannel_nag_hot,nag_city_isholdroom_max,nag_city_isholdroom_var,nag_countryid_isholdroom_mean,nag_countryid_isholdroom_var,pos_hotel_isholdroom_mean,nag_hotel_isholdroom_mean,pos_hotel_isholdroom_min,nag_hotel_isholdroom_min,pos_hotel_isholdroom_max,nag_hotel_isholdroom_max,pos_hotel_isholdroom_var,nag_hotel_isholdroom_var,pos_hotel_isholdroom_median,pos_hotel_isholdroom_mean_sub,pos_hotel_isholdroom_median_sub,nag_hotel_isholdroom_mean_sub,pos_room_isholdroom_mean,pos_room_isholdroom_max,pos_room_isholdroom_median,pos_room_isholdroom_median_sub,pos_masterbasicroomid_isholdroom_median,pos_masterbasicroomid_isholdroom_median_sub,nag_masterhotelid_isholdroom_mean,nag_masterhotelid_isholdroom_min,nag_masterhotelid_isholdroom_var,nag_masterhotelid_isholdroom_median,pos_masterhotelid_isholdroom_median_sub,nag_masterhotelid_isholdroom_mean_sub,nag_masterhotelid_isholdroom_median_sub,pos_supplierid_isholdroom_mean,pos_supplierid_isholdroom_var,pos_supplierid_isholdroom_median,nag_supplierid_isholdroom_mean_sub,nag_city_ordadvanceday_max,pos_city_ordadvanceday_var,pos_city_ordadvanceday_median_sub,pos_countryid_ordadvanceday_median_sub,pos_hotel_ordadvanceday_mean,nag_hotel_ordadvanceday_mean,pos_hotel_ordadvanceday_min,pos_hotel_ordadvanceday_max,nag_hotel_ordadvanceday_max,pos_hotel_ordadvanceday_var,nag_hotel_ordadvanceday_var,pos_hotel_ordadvanceday_median,nag_hotel_ordadvanceday_median,pos_hotel_ordadvanceday_mean_sub,pos_hotel_ordadvanceday_median_sub,nag_hotel_ordadvanceday_mean_sub,nag_hotel_ordadvanceday_median_sub,pos_masterbasicroomid_ordadvanceday_mean,pos_masterbasicroomid_ordadvanceday_median_sub,pos_masterhotelid_ordadvanceday_mean,nag_masterhotelid_ordadvanceday_mean,pos_masterhotelid_ordadvanceday_max,nag_masterhotelid_ordadvanceday_max,nag_masterhotelid_ordadvanceday_var,pos_masterhotelid_ordadvanceday_median,nag_masterhotelid_ordadvanceday_mean_sub,nag_masterhotelid_ordadvanceday_median_sub,pos_supplierid_ordadvanceday_mean,nag_supplierid_ordadvanceday_mean,pos_supplierid_ordadvanceday_max,pos_supplierid_ordadvanceday_var,nag_supplierid_ordadvanceday_var,pos_supplierid_ordadvanceday_median,nag_supplierid_ordadvanceday_median,nag_city_isvendor_median_sub,pos_countryid_isvendor_min,pos_countryid_isvendor_mean_sub,pos_hotel_isvendor_mean,nag_hotel_isvendor_mean,pos_hotel_isvendor_min,pos_hotel_isvendor_max,nag_hotel_isvendor_max,pos_hotel_isvendor_var,nag_hotel_isvendor_var,pos_hotel_isvendor_median,nag_hotel_isvendor_median,pos_hotel_isvendor_mean_sub,pos_hotel_isvendor_median_sub,nag_hotel_isvendor_median_sub,nag_room_isvendor_mean,nag_room_isvendor_min,nag_room_isvendor_max,nag_room_isvendor_median,pos_room_isvendor_mean_sub,pos_room_isvendor_median_sub,nag_room_isvendor_mean_sub,nag_room_isvendor_median_sub,pos_masterhotelid_isvendor_mean,nag_masterhotelid_isvendor_mean,pos_masterhotelid_isvendor_min,pos_masterhotelid_isvendor_max,pos_masterhotelid_isvendor_var,pos_masterhotelid_isvendor_mean_sub,pos_masterhotelid_isvendor_median_sub,nag_masterhotelid_isvendor_mean_sub,pos_supplierid_isvendor_min,nag_supplierid_isvendor_min,nag_supplierid_isvendor_median,pos_city_hotelbelongto_mean,pos_city_hotelbelongto_median,pos_countryid_hotelbelongto_var,pos_countryid_hotelbelongto_mean_sub,pos_countryid_hotelbelongto_median_sub,nag_countryid_hotelbelongto_mean_sub,pos_hotel_hotelbelongto_mean,nag_hotel_hotelbelongto_mean,pos_hotel_hotelbelongto_min,nag_hotel_hotelbelongto_min,nag_hotel_hotelbelongto_max,pos_hotel_hotelbelongto_var,nag_hotel_hotelbelongto_var,pos_hotel_hotelbelongto_median,nag_hotel_hotelbelongto_median,pos_hotel_hotelb

In [7]:
import xgboost as xgb
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score

In [8]:
train[basic_fea] = train[basic_fea].astype(float)

test = train[train.noroom == -1]
train = train[train.noroom != -1]

train.shape
test.shape

(309802, 409)

(22071, 409)

In [9]:
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    p_start = precision_score(labels, (preds>=0.05).astype(int))

    precision, recall, thresholds = precision_recall_curve(labels, preds)
    recall, precision = recall[::-1], precision[::-1]
    area = 0
    had_find_r_start = False
    for idx, item in enumerate(zip(recall, precision)):
        r, p = item[0], item[1]
        if r >= 0.5:
            area += precision[idx-1] * (0.5 - recall[idx-1])
            return 'prc', -area
        if r > 0.05:
            if not had_find_r_start:
                had_find_r_start = True
                area += p_start * (r - 0.05)
            else:
                area += precision[idx-1] * (r - recall[idx-1])
    return 'prc', -area

In [10]:
feature_preformance = {}
params = {
    'objective': 'binary:logistic',
    'eta': 0.1,
    'colsample_bytree': 0.886,
    'min_child_weight': 0.1,
    'max_depth': 15,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'seed': 2018,
    'missing': -1
}
predictors = basic_fea + good_fea[:50]
len(predictors)
xgbtrain = xgb.DMatrix(train[predictors], train[target])

cv = xgb.cv(
params,
dtrain=xgbtrain,
num_boost_round=20000,
nfold=3,
feval=evalerror,
verbose_eval=100,
early_stopping_rounds=100,
seed=2018)
print(str(cv.shape[0]) + ':' + str(cv.iloc[-1, :]['test-prc-mean']))

78

[0]	train-error:0.0354803+0.000432991	train-prc:-0.277843+0.00837847	test-error:0.0426047+0.000662803	test-prc:-0.207173+0.00613792
[100]	train-error:0.019943+0.000327897	train-prc:-0.447652+0.000207497	test-error:0.0395187+0.000541211	test-prc:-0.284696+0.00297494
82:-0.28538899999999995


In [11]:
predictors = basic_fea + good_fea[:50]
len(predictors)
xgbtrain = xgb.DMatrix(train[predictors], train[target])
xgbtest = xgb.DMatrix(test[predictors], train[target])
model = xgb.train(params, xgbtrain, num_boost_round=int(cv.shape[0]*1.2))
pred = model.predict(xgbtest)

407

In [12]:
submit = test[['orderid', 'room', 'arrival']].copy()
submit['noroom'] = pred
submit['arrival'] = submit['arrival'].apply(lambda x:x.replace('/', '-'))
submit['arrival'] = pd.to_datetime(submit['arrival'], format='%Y-%m-%d')
submit['arrival'] = submit['arrival'].dt.strftime('%Y/%m/%d').values
submit = submit.iloc[:11035, :]
submit.to_csv('./submit/xgb_zero210.csv', index=None)

In [13]:
submit.shape
submit.head()

(11035, 4)

,orderid,room,arrival,noroom
309802,4.505897e+09,71340604.0,2017/10/11,0.046458
309803,4.504467e+09,70578904.0,2017/09/19,0.020664
309804,4.505792e+09,91096440.0,2017/09/16,0.021490
309805,4.505284e+09,20487676.0,2017/09/17,0.046505
309806,4.506808e+09,88163053.0,2017/09/17,0.003151
